# Tutorial 2 - AquaHarmonics
The goal of this tutorial is to illustrate a more realistic model of a PTO, including non-linear power conversion chain. 
It uses the [AquaHarmonics](https://aquaharmonics.com/) device in one degree of freedom in regular waves. 
It models the PTO generator using a non-linear efficiency map and adds realistic constraints, including generator maximum torque and min/max line tension.

![AquaHarmonics device](https://aquaharmonics.com/wec_vis.png)

In [ ]:
import pygmsh
import gmsh
import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# import xarray as xr
# from scipy.optimize import brute

import wecopttool as wot

## WEC
Create a WEC: mesh, DOFs, hydrostatics, BEM, constraints

### Mesh
Create a geometry

In [ ]:
T1 = 1.5
T2 = 0.355
T3 = 7.25
r1 = 1.085 
r2 = 0.405 
r3 = 0.355 
offset = 0.1
mesh_size_factor = 0.25

with pygmsh.occ.Geometry() as geom:
    gmsh.option.setNumber('Mesh.MeshSizeFactor', mesh_size_factor)
    cyl1 = geom.add_cylinder([0, 0, 0], [0, 0, -T1], r1)
    cone = geom.add_cone([0, 0, -T1], [0, 0, -T2], r1, r2)
    cylout = geom.add_cylinder([0, 0, -1*(T1+T2)], [0, 0, -T3], r2)
    cylin = geom.add_cylinder([0, 0, -1*(T1+T2)], [0, 0, -T3], r3)
    cyl2 = geom.boolean_difference(cylout, cylin)[0]
    wecGeom = geom.boolean_union([cyl1, cone, cyl2])[0]

    geom.translate(wecGeom, [0, 0, offset])
    mesh = geom.generate_mesh()

In [ ]:
x = [0, r3, r3, r2, r2, r1, r1]
y = [-(T1+T2), -(T1+T2), -(T1+T2+T3), -(T1+T2+T3), -(T1+T2), -T1, offset]
fig, ax = plt.subplots()
ax.plot(x, y, marker='.')
ax.set_xlim(left=0)
ax.axhline(0, color='grey', linestyle='--')
ax.axvline(0, color='grey', linestyle='--')
ax.set_xlabel('Radius [m]')
ax.set_ylabel('Height [m]')
ax.axis('equal')

### Capytaine Floating Body
Add DOFs

In [ ]:
fb = cpy.FloatingBody.from_meshio(mesh, name="AquaHarmonics")
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs

visualize the mesh

In [ ]:
fb.show_matplotlib()

### Mass and hydrostatics

In [ ]:
mass = np.atleast_2d(5500) # [kg] mass w. ballast

# g = 9.81
# rho = 1025
# displaced_mass_cpy = wot.hydrostatics.inertia_matrix(fb).values
# displacement = displaced_mass_cpy/rho # [m^3] disp. vol w. ballast and pretension

stiffness = wot.hydrostatics.stiffness_matrix(fb).values

### frequencies

In [ ]:
f1 = 0.08
nfreq = 10

freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

## PTO
### Main PTO parameters:
- Sprocket radii
- inertia of sprockets
- furction components
- air spring parameters

![PTO diagram](PTO.png)

In [ ]:
radii = {
    "S1": 0.124775, "S2": 0.4991, "S3": 0.1595, "S4": 0.200525, "S5": 0.40105, 
    "S6": 0.12575, "S7": 0.103
}

inertias = {
    "Igen": 3.9, "I1": 0.029, "I2": 25.6, "I3": 1.43, "I4": 1.165, "I5": 4.99, 
    "I6": 1.43, "I7": 1.5, "mps": 40
}

friction = {
    "Bgen": 7, "Bdrivetrain": 40, "Bshaft": 40, "Bspring_pulley": 80, 
    "Bpneumatic_spring": 700, "Bpneumatic_spring_static1": 0, 
    "Bpspneumatic_spring_static2": 0
}

airspring = {
    "gamma": 1.4, "height": 1, "diameter": 3, "area": 0.0709676, 
    "press_init": 854e3, "vol_init": 1
}

gear_ratios = {
    "R21": radii['S2']/radii['S1'],
    "R45": radii['S4']/radii['S5'], 
    "R67": radii['S6']/radii['S7'],
    "spring": radii['S6']*(radii['S4']/radii['S5'])
}

inertia_PTO = (
    (inertias["Igen"]  + inertias["I1"])*gear_ratios['R21']**2 +
    (inertias['I2'] +inertias['I3'] + inertias['I4']) +
    gear_ratios["R45"]**2 * (
        inertias['I5'] + inertias['I6'] +
        inertias["I7"] * gear_ratios['R67']**2 +
        inertias['mps'] * radii['S6']**2   
    )
)

friction_PTO = (
    friction['Bgen']*gear_ratios['R21']**2 + 
    friction['Bdrivetrain'] +
    gear_ratios["R45"]**2 * (
        friction["Bshaft"]+
        friction["Bspring_pulley"]*gear_ratios['R67']**2 +
        friction["Bpneumatic_spring"]*radii['S6']**2
    )
)

### Efficiency map

In [ ]:
def efficiency(flow, effort): 
    eff_max = 300
    flow_max = 10000*2*np.pi/60
    a =       1.148  
    b =     -0.4589  
    c =      -0.297  
    d =       3.204  
    e =      -1.695  
    f =        0.01 
    o =      -3.683 
    efficiency = [((1.2-f/(55*flo**2+0.377)**(3/2))*(1.2-f/(230*eff**2+0.367)**(3/2))
         * ((a**3*flo**2*eff**2) + b**3*eff**2 + c**3.*flo**2 +e**5*eff**4*flo**4 + d) + o) 
         for eff,flo in zip(effort/eff_max,flow/flow_max)]
    return np.array(efficiency)

In [ ]:
rot_max = 10000*2*np.pi/60
torque_max = 300
power_limit = 8000

In [ ]:
x = np.arange(-1*rot_max, 1*rot_max, 10)
y = np.arange(-1*torque_max, 1.0*torque_max, 5)
X, Y = np.meshgrid(x, y)
Z = efficiency(X, Y)
Z[np.abs(X*Y) > power_limit] = np.NaN  # cut off area outside of power limit

# fig, axes = plt.subplots(1, 2, subplot_kw={"projection": "3d"})
fig = plt.figure(figsize=plt.figaspect(0.4))
ax = [
    fig.add_subplot(1, 2, 1, projection="3d"),
    fig.add_subplot(1, 2, 2)
]

ax[0].plot_surface(X, Y, Z, cmap=cm.coolwarm,linewidth=0)
ax[0].set_xlabel('rotational speed [rad/s]')
ax[0].set_ylabel('mechanical torque [Nm]')
ax[0].set_zlabel('efficiency')
ax[0].set_zlim([0, 1.1])

contour = ax[1].contourf(X, Y, Z)
plt.colorbar(contour, label="efficiency")
ax[1].set_xlabel('rotational speed [rad/s]')
ax[1].set_ylabel('mechanical torque [Nm]')

plt.tight_layout()

### Generator

In [ ]:
gear_ratio_generator = gear_ratios['R21']/radii['S3']
gear_ratio_generator = 249.5    
# ??????

### PTO object

In [ ]:
name = ["PTO_Heave",]
kinematics = gear_ratio_generator*np.eye(ndof)
controller = None
nstate_opt = 2*nfreq + 1
pto_impedance = None
pto = wot.pto.PTO(
    ndof, kinematics, controller, pto_impedance, efficiency, name
)

## Additional Forces

In [ ]:
def F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps = 1):
    """Only the zero-th order component (doesn't include linear stiffness"""
    return displacement * rho * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])

def F_gravity(wec, x_wec, x_opt, waves, nsubsteps = 1):
    return -1 * wec.inertia_matrix.item() * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])

def F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps = 1):
    """Pretension force as it acts on the WEC"""
    F_b = F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps) 
    F_g = F_gravity(wec, x_wec, x_opt, waves, nsubsteps)
    return  -1*(F_b+F_g)

def F_pto_passive(wec, x_wec, x_opt, waves, nsubsteps = 1):
    pos = wec.vec_to_dofmat(x_wec)
    vel = np.dot(wec.derivative_mat,pos)
    acc = np.dot(wec.derivative_mat, vel)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    spring = -(gear_ratios['spring']*airspring['gamma']*airspring['area']*
              airspring['press_init']/airspring['vol_init']) * pos
    F_spring = np.dot(time_matrix,spring)
    fric = -(friction_PTO  + 
                friction['Bpneumatic_spring_static1']*
                gear_ratios['spring']) * vel
    F_fric = np.dot(time_matrix,fric)
    inertia = inertia_PTO * acc
    F_inertia = np.dot(time_matrix,inertia)
    return F_spring + F_fric + F_inertia

def F_pto_line(wec, x_wec, x_opt, waves, nsubsteps = 1):
    f_pto = pto.force_on_wec(wec, x_wec, x_opt, waves, nsubsteps)
    f_pre = F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps)
    return f_pto + f_pre

f_add = {'PTO': F_pto_line,
         'PTO_passive': F_pto_passive,
         'buoyancy': F_buoyancy,
         'gravity': F_gravity}

## Generator constraints

In [ ]:
# Generator constraints
torque_peak_max = 280    #[Nm]   
torque_continues_max = 120 #[Nm]
rot_speed_max = 10000*2*np.pi/60    #[rad/s]
power_max = 80000   #[W]
# mooring line constraint
min_line_tension = -1000

nsubsteps = 10

def const_peak_torque_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    """Instantaneous torque must not exceed max torque 
        Tmax - |T| >=0 """
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())

ineq_cons_peak_torque = {'type': 'ineq',
             'fun': const_peak_torque_pto,
             }

def const_torque_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    """RMS torque must not exceed max continous torque 
        Tmax_conti - Trms >=0 """
    torque_rms = np.sqrt(np.mean(pto.force(wec, x_wec, x_opt, waves, nsubsteps)**2))
    return torque_continues_max - np.abs(torque_rms.flatten())

ineq_cons_torque = {'type': 'ineq',
             'fun': const_torque_pto,
             }

def const_speed_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    rot_vel = pto.velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return rot_speed_max - np.abs(rot_vel.flatten())

ineq_cons_rot_speed = {'type': 'ineq',
             'fun': const_speed_pto,
             }

def const_power_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    power_mech = (pto.velocity(wec, x_wec, x_opt, waves, nsubsteps) *
                    pto.force(wec, x_wec, x_opt, waves, nsubsteps))

    return power_max - np.abs(power_mech.flatten())

ineq_cons_power = {'type': 'ineq',
             'fun': const_power_pto,
             }

def constrain_min_tension(wec, x_wec, x_opt, waves):
    total_tension = -1*F_pto_line(wec, x_wec, x_opt, waves, nsubsteps)
    return total_tension.flatten() + min_line_tension

ineq_cons_tension = {'type': 'ineq',
             'fun': constrain_min_tension,
             }

constraints = [
    ineq_cons_tension,
    ineq_cons_torque,
    ineq_cons_peak_torque,
    ineq_cons_rot_speed,
    ineq_cons_power
]


## BEM

In [ ]:

bem_data = wot.run_bem(fb, freq)
# wot.write_netcdf(fname, bem_data)

# scipy.io.savemat('AH_capytaine.mat', 
#                 dict(omega=np.transpose(bem_data['omega'].values),
#                     radiation=bem_data['radiation_damping'].values,
#                     added_mass=bem_data['added_mass'].values,
#                     diffraction = bem_data['diffraction_force'].values,
#                     FK_force =bem_data['Froude_Krylov_force'].values,
#                     stiffness = stiffness,
#                     mass = mass
#                 ))
fig, axes = plt.subplots(3,1)
bem_data['added_mass'].plot(ax = axes[0])
bem_data['radiation_damping'].plot(ax = axes[1])
axes[2].plot(bem_data['omega'],np.abs(np.squeeze(bem_data['diffraction_force'].values)), color = 'orange')
axes[2].set_ylabel('abs(diffraction_force)', color = 'orange')
axes[2].tick_params(axis ='y', labelcolor = 'orange')
ax2r = axes[2].twinx()
ax2r.plot(bem_data['omega'],np.abs(np.squeeze(bem_data['Froude_Krylov_force'].values)), color = 'blue')
ax2r.set_ylabel('abs(FK_force)', color = 'blue')
ax2r.tick_params(axis ='y', labelcolor = 'blue')

### WEC object

In [ ]:
wec = wot.WEC.from_bem(
        bem_data,
        inertia_matrix=mass,
        hydrostatic_stiffness=stiffness,
        constraints=constraints,
        friction=None,
        f_add=f_add,
    )

## Waves

In [ ]:
amplitude = 0.25 
wavefreq =  0.24 #6*f1 
phase = 30
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

## Objective function

In [ ]:
obj_fun = pto.average_power

## Solve

In [ ]:
options = {'maxiter': 200,
           'ftol':1e-8}   #TODO tighter tolerance?



# #scales for unstructured controller A= 0.25m, wf= 0.24 Hz, 10 freq
# scale_x_wec = 1e1*1
# scale_x_opt = 1e-2*50
# scale_obj = 1e-3/1

#scales for unstructured controller A= 0.25m, wf= 0.18 Hz, 10 freq
scale_x_wec = 1e1
scale_x_opt = 1e-2*50
scale_obj = 1e-3*1


results = wec.solve(
    waves, 
    obj_fun, 
    nstate_opt,
    optim_options=options, 
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
    )

print(f'Optimal average power: {results.fun} W')

# post-process
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubsteps=nsubsteps)
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=nsubsteps)

In [ ]:
plt.figure()
pto_tdom['mech_power'].plot()
pto_tdom['power'].plot(linestyle = 'dashed')
plt.plot(wec.time, -1*power_max*np.ones(wec.time.shape),linestyle = 'dotted')



In [ ]:
wec_tdom.force.sel(type='PTO').plot()
# wec_tdom.force.sel(type='PTO_passive').plot(linestyle = 'dotted')
x_wec, x_opt = wot.decompose_state(results.x,ndof=ndof,nfreq=nfreq)
# plt.plot(wec.time, F_pto_line(wec, x_wec, x_opt, waves), linestyle = 'dashed')
plt.plot(wec.time_nsubsteps(nsubsteps), F_pto_line(wec, x_wec, x_opt, waves, nsubsteps), linestyle = 'dashed')
plt.plot(wec.time, min_line_tension*np.ones(wec.time.shape),linestyle = 'dotted')



In [ ]:
total_tension = -1*wec_tdom.force.sel(type='PTO')
constraint_tension =  total_tension + min_line_tension

constraint_tension.plot()

In [ ]:
def align_yyaxis(ax1, ax2):
    ax1_ylims = ax1.axes.get_ylim()           # Find y-axis limits set by the plotter
    ax1_yratio = ax1_ylims[0] / ax1_ylims[1]  # Calculate ratio of lowest limit to highest limit

    ax2_ylims = ax2.axes.get_ylim()           # Find y-axis limits set by the plotter
    ax2_yratio = ax2_ylims[0] / ax2_ylims[1]  # Calculate ratio of lowest limit to highest limit


    # If the plot limits ratio of plot 1 is smaller than plot 2, the first data set has
    # a wider range range than the second data set. Calculate a new low limit for the
    # second data set to obtain a similar ratio to the first data set.
    # Else, do it the other way around

    if ax1_yratio < ax2_yratio: 
        ax2.set_ylim(bottom = ax2_ylims[1]*ax1_yratio)
    else:
        ax1.set_ylim(bottom = ax1_ylims[1]*ax2_yratio)

In [ ]:
# plot
# fine time discretization

fig, axes = plt.subplots(nrows=4,
                       figsize=(6,12), sharex = True,
                       constrained_layout=True)


#velocity and wave excitation force
line1 = wec_tdom.force.sel(type='Froude_Krylov').plot(ax = axes[0], label='FK force')
axes[0].set_ylabel('Force [N]')
axes[0].set_xlabel('')
axes[0].set_title('')
axes[0].text(-0.2, 1, "a)", ha="left", va="top", transform=axes[0].transAxes)

ax0r = axes[0].twinx()
line2 = wec_tdom.vel.plot(ax = ax0r, label = 'WEC velocity', linestyle = 'dashed', color = 'orange')


ax0r.set_ylabel('Velocity [ms] ')
ax0r.set_title('')
ax0r.tick_params(axis='y', color='black', labelcolor='black')
align_yyaxis(axes[0],ax0r)
lines = line1 + line2  
ax0r.legend(lines, ['Excitation force','WEC velocity ', ])

plt.axhline(y=0, xmin = 0, xmax = 1, color = '0.75', linewidth=0.5)
axes[0].grid(color='0.75', linestyle='-',
                     linewidth=0.5, axis = 'x')

# #line tension and PTO force

wec_tdom.force.sel(type='PTO').plot(ax = axes[1], label = 'PTO force in WEC frame')
x_wec, x_opt = wot.decompose_state(results.x,ndof=ndof,nfreq=nfreq)
axes[1].plot(wec.time_nsubsteps(nsubsteps), 
             F_pto_line(wec, x_wec, x_opt, waves, nsubsteps), 
             linestyle = 'dashed', label = 'Mooring line tension')
axes[1].plot(wec.time, min_line_tension*np.ones(wec.time.shape),
            linestyle = 'dotted', color = 'black',
            label = 'Constraint mooring line tension')
axes[1].axhline(y=0, xmin = 0, xmax = 1, color = '0.75', linewidth=0.5)

axes[1].set_title('')
axes[1].set_ylabel('Force [N]')
axes[1].legend()
axes[1].set_xlabel('')
axes[1].grid(color='0.75', linestyle='-',
                     linewidth=0.5, axis = 'x')
axes[1].text(-0.2, 1, "b)", ha="left", va="top", transform=axes[1].transAxes)


# PTO torque in PTO frame

(pto_tdom.force ).plot(ax= axes[2], linestyle = 'solid', label = 'PTO torque in PTO frame')
axes[2].plot(pto_tdom.time, 1*torque_peak_max*np.ones(pto_tdom.time.shape),color = 'black', linestyle = 'dotted', label = 'Constraint peak torque')
axes[2].plot(pto_tdom.time, -1*torque_peak_max*np.ones(pto_tdom.time.shape),color = 'black', linestyle = 'dotted')

torque_rms = np.sqrt(np.mean(pto_tdom.force.values**2))
axes[2].plot(pto_tdom.time, torque_rms*np.ones(pto_tdom.time.shape),linestyle = 'dashed', label = 'RMS(Torque)')
axes[2].plot(pto_tdom.time, torque_continues_max*np.ones(pto_tdom.time.shape), color = 'grey', linestyle = 'dotted', label = 'Constraint continous torque')

axes[2].grid(color='0.75', linestyle='-',
                     linewidth=0.5, axis = 'x')
axes[2].legend(loc = 'upper right',)
axes[2].set_xlabel('')
axes[2].set_ylabel('Torque [Nm] ')

axes[2].set_title('')
axes[2].axhline(y=0, xmin = 0, xmax = 1, color = '0.75', linewidth=0.5)
axes[2].text(-0.2, 1, "c)", ha="left", va="top", transform=axes[2].transAxes)


# Power

pto_tdom['mech_power'].plot(ax = axes[3], label = 'Mechanical power')
pto_tdom['power'].plot(ax = axes[3], linestyle = 'dashed', label = 'Electrical power')
axes[3].plot(wec.time, -1*power_max*np.ones(wec.time.shape),linestyle = 'dotted',color = 'black', label = 'Constraint max power')
axes[3].grid(color='0.75', linestyle='-',
                     linewidth=0.5, axis = 'x')
axes[3].legend()
axes[3].set_title('')
axes[3].axhline(y=0, xmin = 0, xmax = 1, color = '0.75', linewidth=0.5)
axes[3].text(-0.2, 1, "d)", ha="left", va="top", transform=axes[3].transAxes)



fig.savefig(os.path.join(fig_dir,'WEC_TD_results_nonlinear_efficiency.pdf'),
            bbox_inches='tight')

In [ ]:
(pto_tdom.force ).plot(linestyle = 'solid', label = 'Torque')
(pto_tdom.vel ).plot(linestyle = 'solid', label = 'Velocity')


In [ ]:
# fig, axes = plt.subplots(1,1)
# time = pto_tdom.time.values
# axes.plot(time[time<1/wavefreq],pto_tdom['power'].values[time<1/wavefreq])
# mask = [t>1/wavefreq and t < 2/wavefreq for t in time] 
# axes.plot(time[time<1/wavefreq],pto_tdom['power'].values[mask])


In [ ]:
# scipy.io.savemat('AH_orbit.mat', 
#                  dict(time=pto_tdom.time.values,
#                       torque=pto_tdom.force.values,
#                       rot_vel=pto_tdom.vel.values,
# ))

In [ ]:
fig, axes = plt.subplots(1,2)
axes[0].axis('off')
axes[1].axis('off')

axes[0] = fig.add_subplot(1, 2, 1, projection=Axes3D.name)
axes[1] = fig.add_subplot(1, 2, 2, projection=Axes3D.name)
rot_max = 10000*2*np.pi/60
rot_speed = np.arange(-0.9*rot_max, .9*rot_max, 10)
t_max = 280
torque = np.arange(-1.2*t_max, 1.2*t_max, 2)
X, Y = np.meshgrid(rot_speed, torque)
Z = loss_interp(X, Y)
eta = pto_tdom['power']/pto_tdom['mech_power']

Z[np.abs(X*Y) > 80000] = np.NaN
# Plot the loss surface.
surf = axes[0].plot_surface(X, Y,np.minimum(Z,0.25),
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes[0].set_xlabel('gen rotational speed [rad/s]')
axes[0].set_ylabel('mechanical torque [Nm]')
axes[0].set_zlim([0, 1.1])

# Plot the electric power surface.
surf = axes[1].plot_surface(X, Y, X*Y*(1-Z),
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes[1].set_xlabel('gen rotational speed [rad/s]')
axes[1].set_ylabel('mechanical torque [Nm]')
axes[1].set_zlim([-100e3, 100e3])

axes[0].plot3D(np.squeeze(pto_tdom.vel.values), 
          np.squeeze(pto_tdom.force.values),
          np.squeeze(1-eta.values), 'red')
axes[0].plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          loss_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes[0].plot3D(np.zeros(torque.shape), 
          torque,
          loss_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
axes[1].plot3D(np.squeeze(pto_tdom.vel.values), 
          np.squeeze(pto_tdom.force.values),
          np.squeeze(pto_tdom['power'].values+100), 'red')
axes[1].plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          0*loss_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes[1].plot3D(np.zeros(torque.shape), 
          torque,
          0*loss_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
axes[1].view_init(45, -70)

In [ ]:
# Plot the efficiency surface.
eff_surf = 1-loss_interp(X,Y)
eff_surf[np.abs(X*Y) > 80000] = np.NaN  #cut off area outside of power limit
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1, projection=Axes3D.name)
surf = axes.plot_surface(X, Y,eff_surf,
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes.set_xlabel('Generator rotational speed [rad/s]')
axes.set_ylabel('Torque [Nm]')
axes.set_zlabel('Efficiency [ ]')

axes.set_zlim([0.0, 1.1])
axes.plot3D(np.squeeze(pto_tdom.vel.values), 
          np.squeeze(pto_tdom.force.values),
          np.squeeze(eta.values), 'red')
axes.plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          1-loss_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes.plot3D(np.zeros(torque.shape), 
          torque,
          1-loss_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
# axes.view_init(90, -90)
axes.view_init(45, -70)

fig.savefig(os.path.join(fig_dir,'efficiency_orbit_3D.pdf'),
            bbox_inches='tight')


In [ ]:
print(f'Average electrical power {(np.sum(pto_tdom.power.values) * wec.dt/nsubsteps)/wec.tf} W')

## Compare single case to constant efficiency 


In [ ]:
def loss_PE_interp(flow, effort): 
    #Pelec fit with R^2: 0.9991
    # Pelec = [(a*flo*eff)
    #     for eff,flo in zip(effort/eff_max,flow/flow_max)]
    eff_max = 300
    flow_max = 10000*2*np.pi/60
    a =       0.9295  

    efficiency = [(a +eff*0 + flo*0) for eff,flo in zip(effort/eff_max,flow/flow_max)]
    loss = 1-np.array(efficiency)
    return loss

In [ ]:
def constant_PE_interp(flow, effort): 
    #Pelec fit with R^2: 0.9991
    # Pelec = [(a*flo*eff)
    #     for eff,flo in zip(effort/eff_max,flow/flow_max)]
    eff_max = 300
    flow_max = 10000*2*np.pi/60
    a =       0.9295  

    Pe = [(a*eff*flo)*eff_max*flow_max for eff,flo in zip(effort/eff_max,flow/flow_max)]
    Pe =  np.array(Pe)
    return Pe

In [ ]:
#NEW pto
name = ["PTO_Heave",]
kinematics = gear_ratio_generator*np.eye(ndof)
controller = None
nstate_opt = 2*nfreq+1
loss_A = loss_PE_interp
# loss_A = constant_PE_interp
pto_impedance = None
pto_A = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss_A, name)

In [ ]:
def F_pto_line_A(wec, x_wec, x_opt, waves, nsubsteps = 1):
    f_pto = pto_A.force_on_wec(wec, x_wec, x_opt, waves, nsubsteps)
    f_pre = F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps)
    return f_pto + f_pre

In [ ]:
f_add_A = {'PTO': F_pto_line_A,
         'PTO_passive': F_pto_passive,
         'buoyancy': F_buoyancy,
         'gravity': F_gravity}

In [ ]:
def const_peak_torque_pto_A(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    """Instantaneous torque must not exceed max torque 
        Tmax - |T| >=0 """
    torque = pto_A.force(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())
ineq_cons_peak_torque_A = {'type': 'ineq',
             'fun': const_peak_torque_pto_A,
             }
def const_torque_pto_A(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    """RMS torque must not exceed max continous torque 
        Tmax_conti - Trms >=0 """
    torque_rms = np.sqrt(np.mean(pto_A.force(wec, x_wec, x_opt, waves, nsubsteps)**2))
    return torque_continues_max - np.abs(torque_rms.flatten())

ineq_cons_torque_A = {'type': 'ineq',
             'fun': const_torque_pto_A,
             }

def const_speed_pto_A(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    rot_vel = pto_A.velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return rot_speed_max - np.abs(rot_vel.flatten())
ineq_cons_rot_speed_A = {'type': 'ineq',
             'fun': const_speed_pto_A,
             }
def const_power_pto_A(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    power_mech = (pto_A.velocity(wec, x_wec, x_opt, waves, nsubsteps) *
                    pto_A.force(wec, x_wec, x_opt, waves, nsubsteps))

    return power_max - np.abs(power_mech.flatten())
ineq_cons_power_A = {'type': 'ineq',
             'fun': const_power_pto_A,
             }
def constrain_min_tension_A(wec, x_wec, x_opt, waves):
    total_tension = -1*F_pto_line_A(wec, x_wec, x_opt, waves, nsubsteps)
    return total_tension.flatten() + min_line_tension
ineq_cons_tension_A = {'type': 'ineq',
             'fun': constrain_min_tension_A,
             }


constraints_A = [ineq_cons_tension_A,ineq_cons_torque_A, ineq_cons_peak_torque_A, ineq_cons_rot_speed_A, ineq_cons_power_A]

In [ ]:
# update WEC 
wec_A = wot.WEC.from_bem(
            bem_data,
            inertia_matrix= mass,
            hydrostatic_stiffness=stiffness,
            constraints=constraints_A,
            friction=None,
            f_add=f_add_A,
        )

In [ ]:
options_A = {'maxiter': 150,
             'ftol': 1e-7}
obj_fun_A = pto_A.average_power

#scales for unstructured controller, 30 freq
scale_x_wec_A = 1e1
scale_x_opt_A = 1e-2*1
scale_obj_A = 1e-3

results_A = wec_A.solve(
    waves, 
    obj_fun_A, 
    nstate_opt,
    optim_options=options_A, 
    scale_x_wec=scale_x_wec_A,
    scale_x_opt=scale_x_opt_A,
    scale_obj=scale_obj_A,
    )

opt_average_power_A = results_A.fun
print(f'Optimal average power constant efficiency PTO: {opt_average_power_A} W')

In [ ]:
pto_fdom_A, pto_tdom_A = pto.post_process(wec_A, results_A, waves, nsubsteps=nsubsteps)
wec_fdom_A, wec_tdom_A = wec.post_process(results_A, waves, nsubsteps=nsubsteps)

In [ ]:
plt.figure()
pto_tdom_A['mech_power'].plot()
pto_tdom_A['power'].plot(linestyle = 'dashed')
plt.plot(wec_A.time, -1*power_max*np.ones(wec_A.time.shape),linestyle = 'dotted')

In [ ]:
wec_tdom_A.force.sel(type='PTO').plot()
x_wec_A, x_opt_A = wot.decompose_state(results_A.x,ndof=ndof,nfreq=nfreq)
plt.plot(wec_A.time_nsubsteps(nsubsteps), F_pto_line(wec_A, x_wec_A, x_opt_A, waves, nsubsteps), linestyle = 'dashed')
plt.plot(wec_A.time, min_line_tension*np.ones(wec_A.time.shape),linestyle = 'dotted')

In [ ]:
fig_A, axes_A = plt.subplots(2,2)
axes_A[0,0].axis('off')
axes_A[0,1].axis('off')
axes_A[1,0].axis('off')
axes_A[1,1].axis('off')

axes_A[0,0] = fig_A.add_subplot(2, 2, 1, projection=Axes3D.name)
axes_A[0,1] = fig_A.add_subplot(2, 2, 2, projection=Axes3D.name)
axes_A[1,0] = fig_A.add_subplot(2, 2, 3, projection=Axes3D.name)
axes_A[1,1] = fig_A.add_subplot(2, 2, 4, projection=Axes3D.name)

rot_max = 10000*2*np.pi/60
rot_speed = np.arange(-0.7*rot_max, .7*rot_max, 10)
t_max = 280
torque = np.arange(-1.2*t_max, 1.2*t_max, 2)
X, Y = np.meshgrid(rot_speed, torque)

Z = loss_interp(X, Y)
Z_A = loss_PE_interp(X, Y)
eta = pto_tdom['power']/pto_tdom['mech_power']
eta_A = pto_tdom_A['power']/pto_tdom_A['mech_power']

Z[np.abs(X*Y) > 80000] = np.NaN
Z_A[np.abs(X*Y) > 80000] = np.NaN

# Plot the loss surface.
surf = axes_A[0,0].plot_surface(X, Y,Z,
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes_A[0,0].set_xlabel('Rot. speed [rad/s]')
axes_A[0,0].set_ylabel('Torque [Nm]')
axes_A[0,0].set_zlabel('loss [ ]')
axes_A[0,0].set_zlim([0, 1.1])
axes_A[0,0].text2D(0.05, 0.9, "a)", transform=axes_A[0,0].transAxes)


# Plot the electric power surface.
surf = axes_A[0,1].plot_surface(X, Y, X*Y*(1-Z),
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes_A[0,1].set_xlabel('Rot. speed [rad/s]')
axes_A[0,1].set_ylabel('Torque [Nm]')
axes_A[0,1].set_zlim([-100e3, 100e3])
axes_A[0,1].set_zlabel('Electrical power [W]')
axes_A[0,1].text2D(0.05, 0.9, "b)", transform=axes_A[0,1].transAxes)

axes_A[0,0].plot3D(np.squeeze(pto_tdom.vel.values), 
          np.squeeze(pto_tdom.force.values),
          np.squeeze(1-eta.values), 'red')
axes_A[0,0].plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          loss_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes_A[0,0].plot3D(np.zeros(torque.shape), 
          torque,
          loss_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
axes_A[0,1].plot3D(np.squeeze(pto_tdom.vel.values), 
          np.squeeze(pto_tdom.force.values),
          np.squeeze(pto_tdom['power'].values), 'red')
axes_A[0,1].plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          0*loss_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes_A[0,1].plot3D(np.zeros(torque.shape), 
          torque,
          0*loss_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
axes_A[0,1].view_init(45, -70)



# Plot the loss surface.
surf = axes_A[1,0].plot_surface(X, Y,Z_A,
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes_A[1,0].set_xlabel('Rot. speed [rad/s]')
axes_A[1,0].set_ylabel('Torque [Nm]')
axes_A[1,0].set_zlim([0, 1.1])
axes_A[1,0].set_zlabel('loss [ ]')
axes_A[1,0].text2D(0.05, 0.90, "c)", transform=axes_A[1,0].transAxes)

# Plot the electric power surface.
surf = axes_A[1,1].plot_surface(X, Y, X*Y*(1-Z_A),
                       cmap=cm.coolwarm,
                       linewidth=0, alpha = 0.6)
axes_A[1,1].set_xlabel('Rot. speed [rad/s]')
axes_A[1,1].set_ylabel('Torque [Nm]')
axes_A[1,1].set_zlim([-100e3, 100e3])
axes_A[1,1].set_zlabel('Electrical power [W]')

axes_A[1,0].plot3D(np.squeeze(pto_tdom_A.vel.values), 
          np.squeeze(pto_tdom_A.force.values),
          np.squeeze(loss_PE_interp(np.squeeze(pto_tdom_A.vel.values),
                                 np.squeeze(pto_tdom_A.force.values))),
                                  'green', linestyle = 'dotted')
axes_A[1,0].plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          loss_PE_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes_A[1,0].plot3D(np.zeros(torque.shape), 
          torque,
          loss_PE_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
axes_A[1,1].plot3D(np.squeeze(pto_tdom_A.vel.values), 
          np.squeeze(pto_tdom_A.force.values),
          np.squeeze(pto_tdom_A['power'].values), 'green', linestyle = 'dotted')
axes_A[1,1].plot3D(rot_speed, 
          np.zeros(rot_speed.shape),
          0*loss_PE_interp(rot_speed, np.zeros(rot_speed.shape)), 'black', linestyle = 'dashed')
axes_A[1,1].plot3D(np.zeros(torque.shape), 
          torque,
          0*loss_PE_interp(np.zeros(torque.shape), torque), 'black', linestyle = 'dashed')
axes_A[1,1].view_init(45, -70)
axes_A[1,1].text2D(0.05, 0.90, "d)", transform=axes_A[1,1].transAxes)

fig_A.savefig(os.path.join(fig_dir,'loss_orbit_3D.pdf'),
            bbox_inches='tight')


In [ ]:
# Plot the different orbits efficiency surface.

fig, ax = plt.subplots(1,1,
                       figsize=(6,6),)

ax.set_xlabel('Generator rotational speed [rad/s]')
ax.set_ylabel('Torque [Nm]')
ax.grid('on')
ax.plot(np.squeeze(pto_tdom.vel.values), 
          np.squeeze(pto_tdom.force.values),  linestyle = 'dashed', color =  'red', label = 'Nonlinear efficiency map')
ax.plot(np.squeeze(pto_tdom_A.vel.values), 
          np.squeeze(pto_tdom_A.force.values), color = 'green' , label = 'Constant efficiency')
ax.legend()

fig.savefig(os.path.join(fig_dir,'efficiency_orbit_2D.pdf'),
            bbox_inches='tight')

In [ ]:
print(f'f1: {f1}, nfreq: {nfreq}, amplitude: {amplitude}, wavefreq: {wavefreq}, T: {1/wavefreq} \n'
      f'options: {options} \n'  
      f's_x_opt: {scale_x_opt}, s_obj: {scale_obj}, s_x_wec: {scale_x_wec}\n'
      f'n it: {results.nit}, nsubsteps: {nsubsteps}, power: {results.fun:.2f}W')

## Co-optimize mass

In [ ]:
def design_obj_fun(x):
    global n
    n += 1
    # Unpack geometry variables
    mass_var = x[0]
    displacement = displaced_mass_cpy/rho
    #BEM
    fname = os.path.join(results_dir, 'bem.nc')
    bem_data = fname
    #PTO
    name = ["PTO_Heave",]
    kinematics = gear_ratio_generator*np.eye(ndof)
    controller = None
    nstate_opt = 2*nfreq+1
    # loss = loss_interp
    pto_impedance = None
    pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss, name)
    obj_fun = pto.average_power
    #forces
    def F_gravity(wec, x_wec, x_opt, waves, nsubsteps = 1):
        return -1 * wec.inertia_matrix.item() * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])
    def F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps = 1):
        """Only the zero-th order component (doesn't include linear stiffness"""
        return displacement * rho * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])
    def F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps = 1):
        """Pretension force as it acts on the WEC"""
        F_b = F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps) 
        F_g = F_gravity(wec, x_wec, x_opt, waves, nsubsteps)
        return  -1*(F_b+F_g)
    def F_pto_line(wec, x_wec, x_opt, waves, nsubsteps = 1):
        f_pto = pto.force_on_wec(wec, x_wec, x_opt, waves, nsubsteps)
        f_pre = F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps)
        return f_pto + f_pre
    def F_pto_passive(wec, x_wec, x_opt, waves, nsubsteps = 1):
        pos = wec.vec_to_dofmat(x_wec)
        vel = np.dot(wec.derivative_mat,pos)
        acc = np.dot(wec.derivative_mat, vel)
        time_matrix = wec.time_mat_nsubsteps(nsubsteps)
        spring = -(gear_ratios['spring']*airspring['gamma']*airspring['area']*
                airspring['press_init']/airspring['vol_init']) * pos
        F_spring = np.dot(time_matrix,spring)
        fric = -(friction_PTO  + 
                    friction['Bpneumatic_spring_static1']*
                    gear_ratios['spring']) * vel
        F_fric = np.dot(time_matrix,fric)
        inertia = inertia_PTO * acc
        F_inertia = np.dot(time_matrix,inertia)
        return F_spring + F_fric + F_inertia
    #constraints
    torque_peak_max = 280    #[Nm]   
    torque_continues_max = 120 #[Nm]
    rot_speed_max = 10000*2*np.pi/60    #[rad/s]
    power_max = 80000   #[W]
    min_line_tension = -1000
    # nsubsteps = 5

    def const_peak_torque_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
        """Instantaneous torque must not exceed max torque 
            Tmax - |T| >=0 """
        torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps)
        return torque_peak_max - np.abs(torque.flatten())
    ineq_cons_peak_torque = {'type': 'ineq',
                'fun': const_peak_torque_pto,
                }
    def const_torque_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
        """RMS torque must not exceed max continous torque 
            Tmax_conti - Trms >=0 """
        torque_rms = np.sqrt(np.mean(pto.force(wec, x_wec, x_opt, waves, nsubsteps)**2))
        return torque_continues_max - np.abs(torque_rms.flatten())
    ineq_cons_torque = {'type': 'ineq',
                'fun': const_torque_pto,
                }
    def const_speed_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
        rot_vel = pto.velocity(wec, x_wec, x_opt, waves, nsubsteps)
        return rot_speed_max - np.abs(rot_vel.flatten())
    ineq_cons_rot_speed = {'type': 'ineq',
                'fun': const_speed_pto,                }
    def const_power_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
        power_mech = (pto.velocity(wec, x_wec, x_opt, waves, nsubsteps) *
                        pto.force(wec, x_wec, x_opt, waves, nsubsteps))
        return power_max - np.abs(power_mech.flatten())
    ineq_cons_power = {'type': 'ineq',
                'fun': const_power_pto,               }
    def constrain_min_tension(wec, x_wec, x_opt, waves):
        total_tension = -1*F_pto_line(wec, x_wec, x_opt, waves, nsubsteps)
        return total_tension.flatten() + min_line_tension
    ineq_cons_tension = {'type': 'ineq',
                'fun': constrain_min_tension, }

    constraints = [ineq_cons_tension,ineq_cons_torque, ineq_cons_peak_torque, ineq_cons_rot_speed, ineq_cons_power]
     

    f_add_var = {'PTO': F_pto_line,
         'PTO_passive': F_pto_passive,
         'buoyancy': F_buoyancy,
         'gravity': F_gravity}
    # update WEC 
    wec_mass = wot.WEC.from_bem(
                bem_data,
                inertia_matrix= mass_var,
                hydrostatic_stiffness=stiffness,
                constraints=constraints,
                friction=None,
                f_add=f_add_var,
            )

    # Solve
    print(f'Run {n} off {N}: Mass: {mass_var}kg')
    # scale_x_wec = 1e1
    # scale_x_opt = 1e-2*1 
    # scale_obj = 1e-3 
    res = wec_mass.solve(
        reg_wave, 
        obj_fun, 
        nstate_opt, 
        optim_options=options, 
        scale_x_wec=scale_x_wec,
        scale_x_opt=scale_x_opt,
        scale_obj=scale_obj)
    opt_average_power = res.fun
    print(f'Mass: {mass_var}kg, Optimal average power: {opt_average_power} W')
    return res.fun

In [ ]:
#find max mass (when minimum pretension is hit)
min_ten = -1000
max_mass = min_ten/g + displaced_mass_cpy
max_mass

In [ ]:
#mass range over which to search
global n, N
n = 0
N = 7
mass_vals = np.linspace(0.3*max_mass.item(), 1*max_mass.item(), N, endpoint=False)
ranges = (slice(mass_vals[0], mass_vals[-1]+np.diff(mass_vals)[0], np.diff(mass_vals)[0]), )


In [ ]:
regular_waves = []
amplitudes = [0.25, 0.25, 0.5,]
wavefrequencies = [0.16, 0.24, 0.32]
for amplitude, wavefreq in zip(amplitudes, wavefrequencies):
    phase = 30
    wavedir = 0
    single_wave = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

    regular_waves.append(single_wave)
print(f'Wave periods {[1/wf for wf in wavefrequencies]}')

In [ ]:
wavefreq

In [ ]:
logging.getLogger().setLevel(logging.ERROR)

res = []
for iw, reg_wave in enumerate(regular_waves):
    n = 0
    print(f"\n\Wave #: {iw+1}")
    res.append(brute(func=design_obj_fun, ranges=ranges, full_output=True,  finish=None))

In [ ]:
fig, ax = plt.subplots(figsize=(6,3),)
lnstls = ["-","--","-.",":"]
for ir, individ_res in enumerate(res):
    ax.plot(individ_res[2]/max_mass.item(), individ_res[3],  zorder=0,
     label = f'Wave: A={amplitudes[ir]} m, T = {1/wavefrequencies[ir]:.2f}s ',
     linestyle = lnstls[ir])
    ax.scatter(individ_res[2]/max_mass.item(), individ_res[3], zorder=1)
    ax.scatter(individ_res[0]/max_mass.item(), individ_res[1], c='r', zorder=1) #optimum

ax.set_xlabel('Dry mass / max mass, $r_m$ [ ]')
ax.set_ylabel('Average electrical power [W]')
ax.set_title('')
plt.legend()
fig.tight_layout()

fig.savefig(os.path.join(fig_dir,'co_optimization_results.pdf'),
            bbox_inches='tight')

### Re-run optimal cases

In [ ]:
res_opt = []
pto_tdom_opt = []
wec_tdom_opt = []


for individ_res, (index, reg_wave) in zip(res, enumerate(regular_waves)):
    opt_mass = individ_res[0]

    wec_opt = wot.WEC.from_bem(
            bem_data,
            inertia_matrix= opt_mass,
            hydrostatic_stiffness=stiffness,
            constraints=constraints,
            friction=None,
            f_add=f_add,
        )
    res_temp = wec_opt.solve(
        reg_wave, 
        obj_fun, 
        nstate_opt, 
        optim_options=options, 
        scale_x_wec=scale_x_wec,
        scale_x_opt=scale_x_opt,
        scale_obj=scale_obj)
    res_opt.append(res_temp)
    opt_average_power = res_temp.fun
    print(f'Mass: {opt_mass}kg, Optimal average power: {opt_average_power} W') 

    pto_fdom_opt, pto_tdom_temp = pto.post_process(wec_opt, res_temp, regular_waves, nsubsteps=nsubsteps)
    # wec_fdom, wec_tdom_temp = wec.post_process(res_temp, regular_waves, nsubsteps=nsubsteps)
    pto_tdom_opt.append(pto_tdom_temp)
    # wec_tdom_opt.append(wec_tdom_temp)



In [ ]:
fig, axes = plt.subplots(nrows = 3, figsize=(6,9), sharex = False,
                       constrained_layout=True)

lnstls = ["-","--","-.",":"]

for individ_res, pto_tdom, (index, reg_wave) in zip(res, pto_tdom_opt, enumerate(regular_waves)):
    opt_mass = individ_res[0]
    # dt = wec_opt.time[1]
    time = pto_tdom.time
    # slice_start = None# int(nsubsteps/wavefrequencies[index]/dt)
    # slice_end = int(nsubsteps/wavefrequencies[index]/dt*1)
    # print(pto_tdom.time.isel(time=slice(slice_start, slice_end)).values[-1])
    mask = time <= 1/wavefrequencies[index]
    axes[0].plot(time[mask], pto_tdom['vel'].values[mask], 
     linestyle = lnstls[index],
     label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')
    axes[1].plot(time[mask], pto_tdom['force'].values[mask], 
     linestyle = lnstls[index],
     label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')
    axes[2].plot(pto_tdom['vel'].values[mask], pto_tdom['force'].values[mask], 
          linestyle = lnstls[index],
           label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')

    # pto_tdom['vel'].isel(time=slice(slice_start, slice_end)).plot(ax = axes[0], 
    #  linestyle = lnstls[index],
    #  label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')

    # pto_tdom['force'].isel(time=slice(slice_start, slice_end)).plot(ax = axes[1], 
    #  linestyle = lnstls[index],
    #  label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')

    # axes[2].plot(np.squeeze(pto_tdom['vel'].isel(time=slice(slice_start, slice_end)).values), 
    #       np.squeeze(pto_tdom['force'].isel(time=slice(slice_start, slice_end))), 
    #       linestyle = lnstls[index],
    #        label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')



axes[0].grid('on')
axes[0].legend()
axes[0].set_title('')
axes[0].set_ylabel('Generator rotational speed [rad/s]')

axes[1].grid('on')
axes[1].set_title('')
axes[1].set_ylabel('Torque [Nm]')

axes[2].grid('on')
axes[2].set_title('')
axes[2].set_xlabel('Generator rotational speed [rad/s]')
axes[2].set_ylabel('Torque [Nm]')


fig.savefig(os.path.join(fig_dir,'efficiency_single_orbit_2D_optimized.pdf'),
            bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(nrows = 3, figsize=(6,9), sharex = False,
                       constrained_layout=True)

lnstls = ["-","--","-.",":"]

for individ_res, pto_tdom, (index, reg_wave) in zip(res, pto_tdom_opt, enumerate(regular_waves)):
    opt_mass = individ_res[0]
    # dt = wec_opt.time[1]
    time = pto_tdom.time
    # slice_start = None# int(nsubsteps/wavefrequencies[index]/dt)
    # slice_end = int(nsubsteps/wavefrequencies[index]/dt*1)
    # print(pto_tdom.time.isel(time=slice(slice_start, slice_end)).values[-1])
    mask = time <= 2/wavefrequencies[index]
    axes[0].plot(time[mask], pto_tdom['vel'].values[mask], 
     linestyle = lnstls[index],
     label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')
    axes[1].plot(time[mask], pto_tdom['force'].values[mask], 
     linestyle = lnstls[index],
     label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')
    axes[2].plot(pto_tdom['vel'].values[mask], pto_tdom['force'].values[mask], 
          linestyle = lnstls[index],
           label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')

    # pto_tdom['vel'].isel(time=slice(slice_start, slice_end)).plot(ax = axes[0], 
    #  linestyle = lnstls[index],
    #  label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')

    # pto_tdom['force'].isel(time=slice(slice_start, slice_end)).plot(ax = axes[1], 
    #  linestyle = lnstls[index],
    #  label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')

    # axes[2].plot(np.squeeze(pto_tdom['vel'].isel(time=slice(slice_start, slice_end)).values), 
    #       np.squeeze(pto_tdom['force'].isel(time=slice(slice_start, slice_end))), 
    #       linestyle = lnstls[index],
    #        label = f'Mass= {opt_mass:.0f} kg, Wave: A={amplitudes[index]} m, T = {1/wavefrequencies[index]:.2f}s ')



axes[0].grid('on')
axes[0].legend()
axes[0].set_title('')
axes[0].set_ylabel('Generator rotational speed [rad/s]')

axes[1].grid('on')
axes[1].set_title('')
axes[1].set_ylabel('Torque [Nm]')

axes[2].grid('on')
axes[2].set_title('')
axes[2].set_xlabel('Generator rotational speed [rad/s]')
axes[2].set_ylabel('Torque [Nm]')


fig.savefig(os.path.join(fig_dir,'efficiency_double_orbit_2D_optimized.pdf'),
            bbox_inches='tight')